In [ ]:
# --- 1. IMPORTS, SETUPS E INICIALIZAÇÃO ---

import pandas as pd
from pathlib import Path
import sys
import matplotlib.pyplot as plt
import numpy as np

print("--- 1. INICIANDO SETUP ---")

# Adiciona o diretório 'Code' ao path para encontrar o DataUtils
code_dir = Path('..').resolve() / 'Code'
if str(code_dir) not in sys.path:
    sys.path.append(str(code_dir))

import DataUtils as du
from DataUtils import AnalisadorBancario

pd.set_option('display.float_format', lambda x: f'{x:,.2f}')
pd.set_option('display.max_rows', 100) # Para ver mais nomes de bancos
plt.style.use('seaborn-v0_8-whitegrid')

output_dir = Path('..').resolve() / 'Output'
analisador = AnalisadorBancario(diretorio_output=str(output_dir))
print("-" * 50)

In [ ]:
# ==============================================================================
# Gerar Tabela Comparativa a Partir de um Arquivo CSV (DEMORADO)
# ==============================================================================

# Garante que a pasta Results existe
results_dir = Path('.') / 'Results'
results_dir.mkdir(parents=True, exist_ok=True)

# Passo 1: Carregar a lista de CNPJs do seu arquivo CSV
print("--- Lendo a lista de bancos do arquivo Banks.csv ---")
try:
    caminho_csv = Path('.') / 'Banks.csv'
    
    df_bancos_interesse = pd.read_csv(caminho_csv, dtype={'CNPJ': str}, sep=';')
    
    # Agora que a coluna já é string, podemos simplesmente convertê-la para uma lista.
    # Não há mais necessidade de .astype(str) aqui.
    lista_de_bancos_do_csv = df_bancos_interesse['CNPJ'].tolist()
    
    # Verificação adicional: Padronizar cada item da lista para 8 dígitos
    # Isso garante que mesmo que o CSV tenha CNPJs de 14 dígitos, eles serão tratados corretamente.
    # Usaremos a função que já temos no DataUtils para isso!
    lista_de_bancos_do_csv = [du.standardize_cnpj_base8(cnpj) for cnpj in lista_de_bancos_do_csv]
    
    print(f"Encontrados {len(lista_de_bancos_do_csv)} bancos para análise.")
    print("Amostra da lista (padronizada para 8 dígitos):", lista_de_bancos_do_csv[:5])

except FileNotFoundError:
    print(f"ERRO: O arquivo 'Banks.csv' não foi encontrado no diretório 'Analysis/'.")
    lista_de_bancos_do_csv = []

# Só continua se a lista de bancos não estiver vazia
if lista_de_bancos_do_csv:
    # Passo 2: Definir os indicadores
    indicadores_desejados = {
        'Índice de Basileia': {'tipo': 'IFDATA', 'conta': 'Índice de Basileia'},
        'TCB': {'tipo': 'Atributo', 'atributo': 'TCB_IFD_CAD'},
        'Volume de Operações tipo A': {'tipo': 'IFDATA', 'conta': 'A'},
        'Volume de Operações tipo AA': {'tipo': 'IFDATA', 'conta': 'AA'},
        'Volume de Operações tipo B': {'tipo': 'IFDATA', 'conta': 'B'},
        'Volume de Operações tipo C': {'tipo': 'IFDATA', 'conta': 'C'},
        'Volume de Operações tipo D': {'tipo': 'IFDATA', 'conta': 'D'},
        'Volume de Operações tipo E': {'tipo': 'IFDATA', 'conta': 'E'},
        'Volume de Operações tipo F': {'tipo': 'IFDATA', 'conta': 'F'},
        'Volume de Operações tipo G': {'tipo': 'IFDATA', 'conta': 'G'},
        'Volume de Operações tipo H': {'tipo': 'IFDATA', 'conta': 'H'},
        'Total de Volume de Operações': {'tipo': 'IFDATA', 'conta': 'Total Geral'},
        'Depósitos a Prazo': {'tipo': 'IFDATA', 'conta': 'Depósitos a Prazo (a4)'},
        'Letra de Crédito Imobiliário': {'tipo': 'IFDATA', 'conta': 'Letras de Crédito Imobiliário (c1)'},
        'Letra de Crédito Agronegócio': {'tipo': 'IFDATA', 'conta': 'Letras de Crédito do Agronegócio (c2)'},
        'Letras Financeiras': {'tipo': 'IFDATA', 'conta': 'Letras Financeiras (c3)'},
        'RWA': {'tipo': 'IFDATA', 'conta': 'Patrimônio de Referência para Comparação com o RWA'},
        'Ativo Total': {'tipo': 'IFDATA', 'conta': 'Ativo Total'}
        }

    # Passo 3: Chamar a função 'comparar_indicadores'
    print("\n--- Gerando tabela comparativa para os bancos do CSV ---")
    data_analise = 202412

    tabela_final = analisador.comparar_indicadores(
        identificadores=lista_de_bancos_do_csv,
        indicadores=indicadores_desejados,
        data=data_analise,
        fillna=0
    )

    tabela_final['NPL'] = np.where(
        tabela_final['Total de Volume de Operações'] > 0,
        (tabela_final['Volume de Operações tipo G'] + tabela_final['Volume de Operações tipo H']) / tabela_final['Total de Volume de Operações'],
        0
    )

    # Passo 4: Exibir e salvar o resultado
    display(tabela_final)

    try:
        # Salvar o relatório dentro de Results
        caminho_output_tabela = results_dir / f'relatorio_bancos_{data_analise}.xlsx'
        tabela_final.to_excel(caminho_output_tabela)
        print(f"\nRelatório salvo com sucesso em: '{caminho_output_tabela.resolve()}'")
    except Exception as e:
        print(f"\nErro ao salvar o relatório em Excel: {e}")